# **Annex 1: User Acquisition and Sign-ups**

**1. How many times was the app downloaded?**


```
SELECT
	COUNT(DISTINCT app_download_key)
FROM app_downloads
```
Answer: **23608**


**2. How many users signed up on the app?**


```
SELECT
	COUNT(DISTINCT user_id)
FROM signups
```
Answer: **17623**


# **Annex 2: Ride Requests and Completions**

**3. How many rides were requested through the app?**


```
SELECT
	COUNT(DISTINCT ride_id)
FROM ride_requests
```
Answer: **385477**


**4. How many rides were requested and completed through the app?**


```
SELECT
	COUNT(DISTINCT ride_id) AS requested,
    (SELECT
       COUNT(DISTINCT ride_id)
	 FROM ride_requests
     WHERE cancel_ts IS NULL) AS completed
FROM ride_requests
```

Answer:

*   **requested: 385477**
*   **completed: 223652**



**5. How many rides were requested and how many unique users requested a ride?**

```
SELECT
	COUNT(DISTINCT ride_id) AS requested_rides,
    COUNT(DISTINCT user_id) AS unique_users
FROM ride_requests
```
Answer:
* **requested_rides: 385477**
* **unique_users: 12406**


# **Annex 3: Ride Details and Performance**

**6. What is the average time of a ride from pick up to drop off?**


```
SELECT
	AVG(dropoff_ts - pickup_ts) AS avg_time_of_ride
FROM ride_requests
```
Answer: **0 days 00:52:36.738773**


**7. How many rides were accepted by a driver?**


```
SELECT
	COUNT(driver_id)
FROM ride_requests
WHERE driver_id IS NOT NULL
```


Answer: **248379**

# **Annex 4: Payment and Revenue**

**8. How many rides did we successfully collect payments and how much was collected?**


```
SELECT
	COUNT(DISTINCT transaction_id) AS total_rides,
	SUM(purchase_amount_usd) AS total_payments
FROM transactions
WHERE charge_status = 'Approved'
```


Answer:
* **total_rides: 212628**
* **total_payments: 4251667.61**

# **Annex 5: Platform-Specific Analysis**

**9. How many ride requests happened on each platform?**


```
SELECT
	ap.platform,
  COUNT(DISTINCT ride_id) AS total_rides
FROM ride_requests r
FULL JOIN signups s
	ON r.user_id = s.user_id
FULL JOIN app_downloads ap
	ON s.session_id = ap.app_download_key
GROUP BY 1
```

Answer:
* **android: 112317**
* **ios: 234693**
* **web: 38467**

# **Annex 6: User Drop-offs and Conversion Rates**

**10. What is the drop-off from users signing up to users requesting a ride?**


```
SELECT
	100 - (ROUND(((COUNT(DISTINCT user_id)::float)
          /(SELECT
                COUNT(DISTINCT user_id)
            FROM signups))::numeric*100,1)) AS dropp_off
FROM ride_requests
```


Answer: **29.6**

# **Annex 7: Unique User Requests and Completions**

**11. How many unique users requested a ride through the Metrocar app?**


```
SELECT
	COUNT(DISTINCT user_id)
FROM ride_requests
```


Answer: **12406**

**12. How many unique users completed a ride through the Metrocar app?**


```
SELECT
	COUNT(DISTINCT user_id)
FROM ride_requests
WHERE cancel_ts IS NULL
```


Answer: **6233**

# **Annex 8: User-Level Conversion Rates (Percent of Previous Approach)**

**13. Of the users that signed up on the app, what percentage these users requested a ride?**


```
SELECT
	(ROUND(((COUNT(DISTINCT user_id)::float)
          /(SELECT
              COUNT(DISTINCT user_id)
            FROM signups))::numeric*100,1)) AS per_users_who_requested_ride
FROM ride_requests
```


Answer: **70.4**

**14. Of the users that signed up on the app, what percentage these users completed a ride?**


```
SELECT
	(ROUND(((COUNT(DISTINCT user_id)::float)
          /(SELECT
              COUNT(DISTINCT user_id)
            FROM signups))::numeric*100,1))
FROM ride_requests
WHERE cancel_ts IS NULL
```


Answer: **35.4**

**15. Using the "percent of previous" approach, what are the user-level conversion rates for the first 3 stages of the funnel (app download to signup and signup to ride requested)?**

--"percent of previous" approach calculates the conversion rate of each stage relative to the number of users in the previous stage


```
WITH download_signup AS(
SELECT
	(ROUND(((COUNT(DISTINCT user_id)::float)
         /(SELECT
            COUNT(DISTINCT app_download_key)
          FROM app_downloads))::numeric*100,1)) AS app_to_signup
FROM signups
),

signup_ride_requested AS (
 SELECT
	(ROUND(((COUNT(DISTINCT user_id)::float)
        /(SELECT
            COUNT(DISTINCT user_id)
         FROM signups))::numeric*100,1)) AS signup_requested
FROM ride_requests
)

SELECT
	app_to_signup,
	signup_requested
FROM download_signup
CROSS JOIN signup_ride_requested
```


Answer:
* **app_to_signup: 74.6**
* **signup_requested: 70.4**

**16. Using the "percent of previous" approach, what are the user-level conversion rates for the following 3 stages of the funnel?  1. signup, 2. ride requested, 3. ride completed**


```
WITH signup_ride_requested AS (
 SELECT
	(ROUND(((COUNT(DISTINCT user_id)::float)
          /(SELECT
              COUNT(DISTINCT user_id)
           FROM signups))::numeric*100,1)) AS signup_requested
FROM ride_requests
),

requested_completed_ride AS (
SELECT
	(ROUND(((COUNT(DISTINCT user_id)::float)
          /(SELECT
              COUNT(DISTINCT user_id)
           FROM ride_requests))::numeric*100,1)) AS requested_completed_rides
FROM ride_requests
WHERE cancel_ts IS NULL
)

SELECT
	signup_requested,
    requested_completed_rides
FROM signup_ride_requested
CROSS JOIN requested_completed_ride
```
Answer:
* **signup_requested: 70.4**
* **requested_completed_rides: 50.2**

# **Annex 9: User-Level Conversion Rates (Percent of Top Approach)**

**17. Using the "percent of top" approach, what are the user-level conversion rates for the first 3 stages of the funnel(app download to signup and signup to ride requested)?**

-- "percent of top" approach in the context of conversion rates means calculating each stage's conversion rate relative to the number of users at the very top of the funnel



```
WITH download_signup AS(
SELECT
	(ROUND(((COUNT(DISTINCT user_id)::float)
          /(SELECT
              COUNT(DISTINCT app_download_key)
           FROM app_downloads))::numeric*100,1)) AS app_to_signup
FROM signups
),

download_ride_requested AS (
 SELECT
	(ROUND(((COUNT(DISTINCT r.user_id)::float)
          /(SELECT
              COUNT(DISTINCT app_download_key)
           FROM app_downloads))::numeric*100,1)) AS app_requested
FROM ride_requests r
FULL JOIN signups s
	ON r.user_id = s.user_id
FULL JOIN app_downloads ap
	ON s.session_id = ap.app_download_key
)

SELECT
	app_to_signup,
	app_requested
FROM download_signup
CROSS JOIN download_ride_requested
```

Answer:
* **app_to_signup: 74.6**
* **app_requested: 52.5**

**18. Using the "percent of top" approach, what are the user-level conversion rates for the following 3 stages of the funnel? 1. signup, 2. ride requested, 3. ride completed**


```
WITH signup_ride_requested AS (
 SELECT
	(ROUND(((COUNT(DISTINCT user_id)::float)
          /(SELECT
                COUNT(DISTINCT user_id)
            FROM signups))::numeric*100,1)) AS signup_requested
FROM ride_requests
),

signup_completed_ride AS (
SELECT
	(ROUND(((COUNT(DISTINCT user_id)::float)
          /(SELECT
                COUNT(DISTINCT user_id)
            FROM signups))::numeric*100,1)) AS signups_completed_rides
FROM ride_requests
WHERE cancel_ts IS NULL
)

SELECT
	signup_requested,
    signups_completed_rides
FROM signup_ride_requested
CROSS JOIN signup_completed_ride
```


Answer:
* **signup_requested: 70.4**
* **signups_completed_rides: 35.4**